## CC3092 - Deep Learning y Sistemas Inteligentes  
## Hoja de Trabajo 2  
### Semestre II - 2025  

**Integrantes:**  
- Sofía Mishell Velásquez, 22049  
- José Rodrigo Marchena, 22398  
- José Joaquín Campos, 22115  
---

# Ejercicio 1 - Experimentación Práctica

## Task 1 - Preparación del conjunto de datos
En esta sección se carga el conjunto de datos Iris utilizando `sklearn.datasets`. Posteriormente, se dividen los datos en entrenamiento y validación para asegurar que los modelos se prueben adecuadamente en datos no vistos.

In [2]:
# Task 1 - Preparación del conjunto de datos
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Cargar dataset Iris
iris = load_iris()
X, y = iris.data, iris.target

# Normalización
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Separación en train/val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Conversión a tensores
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_val_t   = torch.tensor(X_val, dtype=torch.float32)
y_val_t   = torch.tensor(y_val, dtype=torch.long)

# Crear DataLoaders
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=16, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=16, shuffle=False)

## Task 2 - Arquitectura modelo
Aquí se construye una red neuronal feedforward simple con nn.Module.
La red incluye una capa de entrada, capas ocultas y una capa de salida con número de neuronas igual al número de clases en Iris (3).

In [3]:
# Task 2 - Arquitectura del modelo
import torch.nn as nn

class IrisNet(nn.Module):
    def __init__(self, input_dim=4, hidden_dim=16, output_dim=3):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

# Instanciar modelo
model = IrisNet()
print(model)

IrisNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)


## Task 3 - Funciones de Pérdida
El objetivo es comparar el rendimiento del modelo utilizando distintas funciones de pérdida.
Se probarán al menos tres pérdidas comunes:
- nn.CrossEntropyLoss (adecuada para clasificación multiclase)
- nn.MSELoss (Mean Squared Error)
- nn.NLLLoss (Negative Log Likelihood, usando LogSoftmax)

In [4]:
# Task 3 - Funciones de Pérdida
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Diccionario de funciones de pérdida
loss_functions = {
    "cross_entropy": nn.CrossEntropyLoss(),
    "mse": nn.MSELoss(),
    "nll": nn.NLLLoss()
}

# Entrenamiento parametrizable
def train_model(loss_name="cross_entropy", use_l1=False, use_l2=False, epochs=50):
    model = IrisNet()
    
    # Selección de función de pérdida
    if loss_name == "nll":
        criterion = loss_functions["nll"]
        model_output = nn.LogSoftmax(dim=1)
    else:
        criterion = loss_functions[loss_name]
        model_output = lambda x: x  # salida directa
    
    # Optimizador con L2 si se requiere
    optimizer = optim.Adam(model.parameters(), lr=0.01,
                           weight_decay=1e-4 if use_l2 else 0.0)
    
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            outputs = model_output(model(xb))
            
            # MSE requiere one-hot encoding
            if loss_name == "mse":
                yb_onehot = F.one_hot(yb, num_classes=3).float()
                loss = criterion(outputs, yb_onehot)
            else:
                loss = criterion(outputs, yb)
            
            # Penalización L1
            if use_l1:
                l1_penalty = sum(param.abs().sum() for param in model.parameters())
                loss += 1e-4 * l1_penalty

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Validación
        model.eval()
        with torch.no_grad():
            val_losses = []
            for xb, yb in val_loader:
                val_out = model_output(model(xb))
                if loss_name == "mse":
                    yb_onehot = F.one_hot(yb, num_classes=3).float()
                    val_loss = criterion(val_out, yb_onehot)
                else:
                    val_loss = criterion(val_out, yb)
                val_losses.append(val_loss.item())
        
        if (epoch+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] "
                  f"| Train Loss: {loss.item():.4f} "
                  f"| Val Loss: {sum(val_losses)/len(val_losses):.4f}")
    
    return model

# Ejemplos de entrenamiento con diferentes funciones de pérdida
print("== Entrenando con CrossEntropy ==")
train_model("cross_entropy")

print("\n== Entrenando con MSE ==")
train_model("mse")

print("\n== Entrenando con NLL ==")
train_model("nll")


== Entrenando con CrossEntropy ==
Epoch [10/50] | Train Loss: 0.1757 | Val Loss: 0.0911
Epoch [20/50] | Train Loss: 0.2706 | Val Loss: 0.0758
Epoch [30/50] | Train Loss: 0.0006 | Val Loss: 0.0844
Epoch [40/50] | Train Loss: 0.0347 | Val Loss: 0.1196
Epoch [50/50] | Train Loss: 0.0025 | Val Loss: 0.1140

== Entrenando con MSE ==
Epoch [10/50] | Train Loss: 0.0602 | Val Loss: 0.0344
Epoch [20/50] | Train Loss: 0.0518 | Val Loss: 0.0264
Epoch [30/50] | Train Loss: 0.0441 | Val Loss: 0.0225
Epoch [40/50] | Train Loss: 0.0012 | Val Loss: 0.0215
Epoch [50/50] | Train Loss: 0.0014 | Val Loss: 0.0229

== Entrenando con NLL ==
Epoch [10/50] | Train Loss: 0.1294 | Val Loss: 0.1065
Epoch [20/50] | Train Loss: 0.0443 | Val Loss: 0.0859
Epoch [30/50] | Train Loss: 0.0014 | Val Loss: 0.1014
Epoch [40/50] | Train Loss: 0.0023 | Val Loss: 0.0893
Epoch [50/50] | Train Loss: 0.0009 | Val Loss: 0.1270


IrisNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)